# Artificial Intelligence Engineer Nanodegree - Probabilistic Models
## Project: Sign Language Recognition System
- [Introduction](#intro)
- [Part 1 Feature Selection](#part1_tutorial)
    - [Tutorial](#part1_tutorial)
    - [Features Submission](#part1_submission)
    - [Features Unittest](#part1_test)
- [Part 2 Train the models](#part2_tutorial)
    - [Tutorial](#part2_tutorial)
    - [Model Selection Score Submission](#part2_submission)
    - [Model Score Unittest](#part2_test)
- [Part 3 Build a Recognizer](#part3_tutorial)
    - [Tutorial](#part3_tutorial)
    - [Recognizer Submission](#part3_submission)
    - [Recognizer Unittest](#part3_test)
- [Part 4 (OPTIONAL) Improve the WER with Language Models](#part4_info)

<a id='intro'></a>
## Introduction
* **Goal** - build a word recognizer for American Sign Language (ASL) video sequences, demonstrating the power of probabalistic models.  
* **Approach** - [Hidden Markov models (HMM's)](https://en.wikipedia.org/wiki/Hidden_Markov_model) to analyze a series of measurements taken from ASL videos collected for research (see the [RWTH-BOSTON-104 Database](http://www-i6.informatik.rwth-aachen.de/~dreuw/database-rwth-boston-104.php)).  In the video shown, the right-hand x and y locations are plotted as the speaker produces sign language (signs) the sentence.
[![ASLR demo](http://www-i6.informatik.rwth-aachen.de/~dreuw/images/demosample.png)](https://drive.google.com/open?id=0B_5qGuFe-wbhUXRuVnNZVnMtam8)

* **Inputs Pre-defined**: raw data, train, and test sets
* **Part 1**: Derive variety of feature sets. 
* **Part 2**: Implement 3x different model selection criterion to determine the optimal number of hidden states for each word model. 
* **Part 3**: Implement recognizer. Compare effects of the different combinations of feature sets and model selection criteria.  
* **Outputs**: At the end of each Part, complete the submission cells with implementations, answer all questions, pass all unit tests, then submit completed notebook for review.

<a id='part1_tutorial'></a>
## PART 1: Data

### Features Tutorial
##### Load the initial database
* **Inputs** - In **asl_data.py** (module) a **Data Handler** designed for this database is in the `AslDb` class.  It creates the initial [pandas](http://pandas.pydata.org/pandas-docs/stable/) dataframe from data included in the `data` directory as well as dictionaries suitable for extracting data in a format friendly to the [hmmlearn](https://hmmlearn.readthedocs.io/en/latest/) library, which are used to create models in Part 2.

First set up initial database. Select an example set of features for the training set.  At the end of Part 1, you will create additional feature sets for experimentation. 

In [ ]:
###_DEBUG_IDE

# # Auto-reload all changed modules every time before executing a new line
# %load_ext autoreload 
# %autoreload 2

def test():
    print("Called .ipynb file from .py file using notebook_importing.py")

import numpy as np
import pandas as pd
from asl_data import AslDb

# Initializes the database
asl = AslDb()

# Display first five rows (pandas data frame) of the ASL database, 
# indexed by video and frame

# print(asl.df.__dict__)
"""
{  'is_copy': None, '_data': 
    BlockManager Items: 
        Index(['left-x', 'left-y', 'right-x', 'right-y', 'nose-x', 'nose-y', 'speaker'], 
          dtype='object')
    Axis 1: MultiIndex(levels=[[1, 2, 3, .., 200, 201], 
                               [0, 1, 2, ..., 160, 161]],
                       labels=[[97, 97, ..., 124, 124], 
                               [0, 1, 2,  56]], 
                       names=['video', 'frame'])
    IntBlock: slice(0, 6, 1), 6 x 15746, dtype: int64
    ObjectBlock: slice(6, 7, 1), 1 x 15746, dtype: object, '_item_cache': {}
}
"""

asl.df.head()


In [ ]:
# Display data available for an individual frame
# Reference: `asl_utils.py`

asl.df.ix[98,1]

The frame represented by video 98, frame 1 is shown here:
![Video 98](http://www-i6.informatik.rwth-aachen.de/~dreuw/database/rwth-boston-104/overview/images/orig/098-start.jpg)

##### Frame represented by video 150

In [ ]:
# Display data available for an individual frame
# Reference: `asl_utils.py`

asl.df.ix[150,1]

The frame represented by video 150, frame 1 is shown here:
![Video 150](http://www-i6.informatik.rwth-aachen.de/~dreuw/database/rwth-boston-104/overview/images/orig/150-start.jpg)

##### Feature selection for training the model
* About: Feature Selection when training a model
* Objectives: 
    * Choose most relevant variables.
    * Keeping model as simple as possible to reduce training time.  
* Inputs: 
    * Raw features (already provided)
    * Custom derivation of our own. Add columns to pandas dataframe `asl.df` for selection.  

In [ ]:
# Calculate the feature named 'grnd-ry' ("ground" right y value), which is 
# the difference b/w right-hand y value and nose y value. 
# Add it to the `asl.df` frames dictionary
# (append result as column label/values to the right of Table) data frame 

asl.df['grnd-ry'] = asl.df['right-y'] - asl.df['nose-y']
asl.df.head()

##### Run

* Click the code snippet cell below and select "Run All"

### Task DONE

* Add df columns for 'grnd-rx', 'grnd-ly', 'grnd-lx' representing differences between hand and nose locations

In [ ]:
from asl_utils import test_features_tryit

# DONE 
# Add df columns for 'grnd-rx', 'grnd-ly', 'grnd-lx' representing 
# differences between hand and nose locations
asl.df['grnd-ry'] = asl.df['right-y'] - asl.df['nose-y']
asl.df['grnd-rx'] = asl.df['right-x'] - asl.df['nose-x']
asl.df['grnd-ly'] = asl.df['left-y'] - asl.df['nose-y']
asl.df['grnd-lx'] = asl.df['left-x'] - asl.df['nose-x']

asl.df.head()

# Test the code
# Expected: [9, 113, -12, 119]
# test_features_tryit(asl)

In [ ]:
# Collect the features into a list
features_ground = ['grnd-rx','grnd-ry','grnd-lx','grnd-ly']

# Show a single set of features for a given (video, frame) tuple
[asl.df.ix[98,1][v] for v in features_ground]

##### Build the training set

* Collect the feature for all the Words in the training set by passing the defined feature list `features_ground` to the `AslDb.build_training` method of `asl_data.py`. Each WordData object (see `WordsData` class in `asl_data.py` in the training set has multiple examples from various videos suitable for the hmmlearn library. 
* WordData object is dictionary of lists of feature list sequence lists for each word `{'FRANK': [[[87, 225], [87, 225], ...], [[88, 219], [88, 219], ...]]]}`.
* Shown below are unique words that have been loaded into the training set:

In [ ]:
training = asl.build_training(features_ground)

print("Training words - num_items: {}".format(training.num_items))
print("Training words - words: {}".format(training.words))

* Training data object `training` is an instance of class `WordsData` built by calling the `build_training` method of class `AslDb` defined in the `asl_data.py` module. Access data using:
    * `WordsData` instance properties:
        * `words` - get list of keys
    * `WordsData` instance methods:
        * `get_all_sequences` - getter for entire db of words as series of sequences of feature lists for each frame
        * `get_all_Xlengths` - getter for entire db of words as (X, lengths) tuple for use training multiple sequences with the `hmmlearn` library. where X is a numpy array of feature lists and lengths is list of lengths of sequences within X. i.e. `{'FRANK': (array([[ 87, 225],[ 87, 225], ... [ 87, 225,  62, 127], [ 87, 225,  65, 128]]), [14, 18]), }`
        * `get_word_sequences` - getter for single word series of sequences of feature lists for each frame
        * `get_word_Xlengths` - getter for single word (X, lengths) tuple for use with hmmlearn library, where X is a numpy array of feature lists and lengths is a list of lengths of sequences within X

* The following example outputs two nested lists:
    * First list is a concatenation of all the sequences(the X portion)
    * Second is a list of the sequence lengths(the Lengths portion).

In [ ]:
training.get_word_Xlengths('CHOCOLATE')

###### More feature sets
* `asl.df` is still a Simple "feature set", but is enough to started modeling.
* Improve results fom `asl.df` by manipulating the raw values. Below we setup other options for later experimentation. 
    * Option 1: **Grouped Statistics** to normalize each speaker's range of motion using [Pandas stats](http://pandas.pydata.org/pandas-docs/stable/api.html#api-dataframe-stats) functions and [pandas groupby](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.groupby.html). See below example for finding the Means of all speaker subgroups.

In [ ]:
df_means = asl.df.groupby('speaker', 0, None, True, True, True, False).mean()
df_means

To select a mean that matches by speaker, use the pandas [map](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.Series.map.html) method:

In [ ]:
# asl.df['left-x-mean']= asl.df['speaker'].map(df_means['left-x'], na_action=None)
asl.df.head()

##### Run

* Click the code snippet cell below and select "Run All"

### Task DONE

* Create a dataframe named `df_std` with standard deviations grouped by speaker

In [ ]:
from asl_utils import test_std_tryit
# DONE Create a dataframe named `df_std` with standard deviations grouped by speaker

df_std = asl.df.groupby('speaker', 0, None, True, True, True, False).std()

asl.df['left-x-mean'] = asl.df['speaker'].map(df_means['left-x'], na_action=None)
asl.df['left-x-std'] = asl.df['speaker'].map(df_std['left-x'], na_action=None)
asl.df['left-y-mean'] = asl.df['speaker'].map(df_means['left-y'], na_action=None)
asl.df['left-y-std'] = asl.df['speaker'].map(df_std['left-y'], na_action=None)

asl.df['right-x-mean'] = asl.df['speaker'].map(df_means['right-x'], na_action=None)
asl.df['right-x-std'] = asl.df['speaker'].map(df_std['right-x'], na_action=None)
asl.df['right-y-mean'] = asl.df['speaker'].map(df_means['right-y'], na_action=None)
asl.df['right-y-std'] = asl.df['speaker'].map(df_std['right-y'], na_action=None)

asl.df.head()

In [ ]:
# test the code
test_std_tryit(df_std)

<a id='part1_submission'></a>
### Features Implementation Submission
Implement four feature sets and answer the question that follows.
- normalized Cartesian coordinates
    - use *mean* and *standard deviation* statistics and the [standard score](https://en.wikipedia.org/wiki/Standard_score) equation to account for speakers with different heights and arm length
    
```
Standard Score (of a raw score x)

z = x - m / std

where m is the mean
where std is the standard deviation
```

- polar coordinates
    - calculate polar coordinates with [Cartesian to polar equations](https://en.wikipedia.org/wiki/Polar_coordinate_system#Converting_between_polar_and_Cartesian_coordinates)
    - use the [np.arctan2](https://docs.scipy.org/doc/numpy-1.10.0/reference/generated/numpy.arctan2.html) function and *swap the x and y axes* to move the $0$ to $2\pi$ discontinuity to 12 o'clock instead of 3 o'clock;  in other words, the normal break in radians value from $0$ to $2\pi$ occurs directly to the left of the speaker's nose, which may be in the signing area and interfere with results.  By swapping the x and y axes, that discontinuity move to directly above the speaker's head, an area not generally used in signing.

- delta difference
    - as described in Thad's lecture, use the difference in values between one frame and the next frames as features. Videos number 12 and 13 briefly mention using delta-y as a feature. The idea of using delta y as a feature is that the direction (i.e. positive or negative y) and the speed of movement between successive frames could be more useful than where the hand is exactly. delta values for the first frame of each video should be zero.
    - pandas [diff method](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.diff.html) and [fillna method](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.fillna.html) will be helpful for this one

- custom features
    - These are your own design; combine techniques used above or come up with something else entirely. We look forward to seeing what you come up with! 
    Some ideas to get you started:
        - normalize using a [feature scaling equation](https://en.wikipedia.org/wiki/Feature_scaling)
        - normalize the polar coordinates
        - adding additional deltas


In [ ]:
# DONE add features for normalized by speaker values of left, right, x, y
# Name these 'norm-rx', 'norm-ry', 'norm-lx', and 'norm-ly'
# using Z-score scaling (X-Xmean)/Xstd 

# Standard Score equation to normalise Cartesian coordinates x and y
def standard_score(x, m, std):
    return (x - m) / std

features_r_l = ['right-x', 'right-y', 'left-x', 'left-y']

features_norm = ['norm-rx', 'norm-ry', 'norm-lx', 'norm-ly']

for f_index, f_val in enumerate(features_norm):
    f_mean = asl.df['speaker'].map(df_means[features_r_l[f_index]], na_action=None)
    f_std  = asl.df['speaker'].map(df_std[features_r_l[f_index]], na_action=None)
    asl.df[f_val] = standard_score(asl.df[features_r_l[f_index]], f_mean, f_std)

asl.df.head()

In [ ]:
# DONE add features for polar coordinate values where the nose is the origin
# Name these 'polar-rr', 'polar-rtheta', 'polar-lr', and 'polar-ltheta'
# Note that 'polar-rr' and 'polar-rtheta' refer to the radius and angle

# Convert Cartesian coordinates x and y to polar coords with r ≥ 0 and ϕ (phi) in the interval (−pi, pi] by:
#
# radius = sqrt(x^2 + y^2)

# Use the np.arctan2 function for tan-1 ( y / x ) and swap the x and y axes to move the 0 to 2pi discontinuity to 12 o'clock 
# instead of 3 o'clock (i.e. the normal break in radians value from 0 to 2pi occurs directly to the left 
# of the speaker's nose, which may be in the signing area and interfere with results, but by swapping 
# the x and y axes, that discontinuity moves to directly above the speaker's head area that's not generally
# used in signing.
#
# angle = np.arctan2(y, x)   

# Convert Cartesian coordinates x and y to radius (Euclidean Norm) in Polar Coordinates 
def radius(x, y):
    return np.sqrt(x**2 + y**2)

# Convert Cartesian coordinates x and y to angle (theta) in Polar Coordinates 
def angle(x, y):
    # Swap the x and y axes in the calculation
    # (i.e. instead of `np.arctan2(y, x)`)
    return np.arctan2(x, y)

# Differences between hand and nose locations (see previous task where calculated)
features_ground = ['grnd-rx', 'grnd-ry', 'grnd-lx', 'grnd-ly']

features_polar = ['polar-rr', 'polar-rtheta', 'polar-lr', 'polar-ltheta']

for f_index, f_val in enumerate(features_polar):
    if f_index % 2 == 0: # even indexes (i.e. 0, 2)
        asl.df[f_val] = radius(asl.df[features_ground[f_index]], 
                               asl.df[features_ground[f_index + 1]])
    else: # odd indexes
        asl.df[f_val] = angle(asl.df[features_ground[f_index - 1]], 
                              asl.df[features_ground[f_index]])
asl.df.head()

In [ ]:
# DONE add features for left, right, x, y differences by one time step, i.e. the "delta" values discussed in the lecture
# Name these 'delta-rx', 'delta-ry', 'delta-lx', and 'delta-ly'

# Replace any NaN values with 0 after taking the results of the difference
def frame_difference(f):
    return f.diff().fillna(0, None, 0, False, None, None)

features_r_l = ['right-x', 'right-y', 'left-x', 'left-y']

# Features representing the difference in values between one frame and the next frames 
features_delta = ['delta-rx', 'delta-ry', 'delta-lx', 'delta-ly']

for f_index, f_val in enumerate(features_delta):
    asl.df[f_val] = frame_difference(asl.df[features_r_l[f_index]])

asl.df.head()

In [ ]:
# DONE add features of your own design, which may be a combination of the above or something else
# Name these whatever you would like, such as:
#   - normalize using a Feature Scaling equation

# Rescaling the range of features to scale to the range in [0, 1] or [-1, 1]
# using the Feature Scaling equation to normalise:
#
#   normalised = orig - min(orig) / max(orig) - min(orig)
#
def rescale(orig, min, max):
    return (orig - min) / (max - min)

df_max = asl.df.groupby('speaker').max()
df_min = asl.df.groupby('speaker').min()

# Differences between hand and nose locations (see previous task where calculated)
features_ground = ['grnd-rx', 'grnd-ry', 'grnd-lx', 'grnd-ly']

# Features representing the Max and Min 
features_max = ['right-x-max', 'right-y-max', 'left-x-max', 'left-y-max']
features_min = ['right-x-min', 'right-y-min', 'left-x-min', 'left-y-min']

features_rescaled = ['right-x-rescaled', 'right-y-rescaled', 'left-x-rescaled', 'left-y-rescaled']

for f_index, f_val in enumerate(features_ground):

    # Map the Max and Min onto the data
    asl.df[features_max[f_index]] = asl.df['speaker'].map(df_max[f_val], na_action=None)
    asl.df[features_min[f_index]] = asl.df['speaker'].map(df_min[f_val], na_action=None)
    
    # Normalise by rescaling using the Feature Scaling equation
    asl.df[features_rescaled[f_index]] = rescale(asl.df[f_val],
                                               asl.df[features_min[f_index]],
                                               asl.df[features_max[f_index]])

# DONE define list named 'features_custom' for building the training set
features_custom = ['right-x-rescaling', 'right-y-rescaling',  'left-x-rescaling', 'left-y-rescaling']

asl.df.head()

**Question 1:**  What custom features did you choose for the features_custom set and why?

**Answer 1:** The custom feature chosen for the features_custom was to rescale the features into a lower range between 0 and 1 for the Cartesian coordinates x and y use the Feature Scaling equation to achieve the normalisation. 

The benefits of rescaling are that interpretation and analysis of plotted data becomes easier. In stochastic environments such as the one we are using, rescaling is known to improve the convergence speed of gradient-descent. We are using similar gradient-based optimisation methods such as the iterative Baum-Welch algorithm (aka Expectation-Maximization (EM) algorithm) to solve the third fundamental problem of HMMs by estimating the model parameters given just the observed data. Rescaling is therefore important since HMMs already have a large vocabulary of signs and use a lot of memory.


<a id='part1_test'></a>
### Features Unit Testing
Run the following unit tests as a sanity check on the defined "ground", "norm", "polar", and 'delta"
feature sets.  The test simply looks for some valid values but is not exhaustive.  However, the project should not be submitted if these tests don't pass.

In [ ]:
# Verify values of features before tests

# Collect the features into a list
features_ground = ['grnd-rx','grnd-ry','grnd-lx','grnd-ly']
features_norm = ['norm-rx', 'norm-ry', 'norm-lx', 'norm-ly']
features_polar = ['polar-rr', 'polar-rtheta', 'polar-lr', 'polar-ltheta']
features_delta = ['delta-rx', 'delta-ry', 'delta-lx', 'delta-ly']

# Show a single set of features for a given (video, frame) tuple
print([asl.df.ix[98,1][v] for v in features_ground])
print([asl.df.ix[98,1][v] for v in features_norm])
print([asl.df.ix[98,1][v] for v in features_polar])
print([asl.df.ix[98,1][v] for v in features_delta])

In [ ]:
import unittest
# import numpy as np

class TestFeatures(unittest.TestCase):

    def test_features_ground(self):
        sample = (asl.df.ix[98, 1][features_ground]).tolist()
        self.assertEqual(sample, [9, 113, -12, 119])

    def test_features_norm(self):
        sample = (asl.df.ix[98, 1][features_norm]).tolist()
        np.testing.assert_almost_equal(sample, [ 1.153,  1.663, -0.891,  0.742], 3)

    def test_features_polar(self):
        sample = (asl.df.ix[98,1][features_polar]).tolist()
        np.testing.assert_almost_equal(sample, [113.3578, 0.0794, 119.603, -0.1005], 3)

    def test_features_delta(self):
        sample = (asl.df.ix[98, 0][features_delta]).tolist()
        self.assertEqual(sample, [0, 0, 0, 0])
        sample = (asl.df.ix[98, 18][features_delta]).tolist()
        self.assertTrue(sample in [[-16, -5, -2, 4], [-14, -9, 0, 0]], "Sample value found was {}".format(sample))
                         
suite = unittest.TestLoader().loadTestsFromModule(TestFeatures())
unittest.TextTestRunner().run(suite)

<a id='part2_tutorial'></a>
## PART 2: Model Selection
### Model Selection Tutorial
* Objective: Model Selection should tune the number of states for each word HMM prior to testing on unseen data. 3x tuning methods include: 
    * Log likelihood using Cross-Validation folds (CV)
    * Bayesian Information Criterion (BIC)
    * Discriminative Information Criterion (DIC) 

##### Train a single word
* "train" models for each word using the training set we built with sequence data. 
* e.g. Train a single word using Gaussian hidden Markov models (HMM) using the `fit` method during training. [Baum-Welch Expectation-Maximization](https://en.wikipedia.org/wiki/Baum%E2%80%93Welch_algorithm) (EM) algorithm is invoked iteratively to find the best estimate for the model (for the number of hidden states specified) from a group of sample sequences. 
    * Assume the correct number of hidden states is 3 (a guess). The "best" number of states for training is determined by some model selection technique that chooses the "best" parameter.

In [ ]:
import warnings
from hmmlearn.hmm import GaussianHMM

def train_a_word(word, num_hidden_states, features):
    
    warnings.filterwarnings("ignore", category=DeprecationWarning)
    
    # Build training set
    training = asl.build_training(features)
    
    # Call getter for single word (X, lengths) tuple for use with hmmlearn library,
    # where "X" is a numpy array of feature lists
    # where "lengths" is a list of lengths of sequences within X
    X, lengths = training.get_word_Xlengths(word)
    
    # Build instance of Hidden Markov Model (HMM) with Gaussian emissions
    # - "fit" method trains the HMM using the input matrix of concatenated sequences
    #   of observations (aka samples), and the lengths of each seqence.
    # - "n_components" attribute is the number of hidden States in the HMM
    # - "n_iter" is number of Baum-Welch (EM algorithm) iterations
    #
    # Reference: 
    # - http://hmmlearn.readthedocs.io/en/latest/tutorial.html
    # - http://hmmlearn.readthedocs.io/en/latest/api.html
    model = GaussianHMM(n_components=num_hidden_states, n_iter=1000).fit(X, lengths)
    
    # "score" method calculates the log likelihood score of the HMM sample(s). 
    # Since the Baum-Welch algorithm
    # (aka Expectation-Maximization (EM) algorithm) is a gradient-based optimisation method
    # we try to avoid it getting stuck in local optima but running the "fit" method with
    # different initialisations, and then selecting the highest scored model.
    logL = model.score(X, lengths)
    return model, logL

demoword = 'BOOK'
model, logL = train_a_word(demoword, 3, features_ground)
print("Built instance of a Hidden Markov Model (HMM) with Gaussian emissions using hmmlearn")
print("Number of states trained in model for {} is {}".format(demoword, model.n_components))
print("logL = {}".format(logL))

The HMM model has been trained and information can be pulled from the model, including means and variances for each feature and hidden state.  The [log likelihood](http://math.stackexchange.com/questions/892832/why-we-consider-log-likelihood-instead-of-likelihood-in-gaussian-distribution) for any individual sample or group of samples can also be calculated with the `score` method.

In [ ]:
def show_model_stats(word, model):
    print("Number of states trained in model for word {} is {}\n".format(word, model.n_components))    
    variance = np.array([np.diag(model.covars_[i]) for i in range(model.n_components)])    
    for i in range(model.n_components):  # for each hidden state
        print("\t - Hidden state #{}".format(i))
        print("\t - Mean = ", model.means_[i])
        print("\t - Variance = ", variance[i])
        print()
    
show_model_stats(demoword, model)

##### Try it!
Experiment by changing the feature set, word, and/or num_hidden_states values in the next cell to see changes in values.  

In [ ]:
# Replace with any word key contained in `training.words`
# (i.e. CHOCOLATE, BOOK, etc)
my_testword = 'CHOCOLATE'

# Experiment here with different parameters
# (i.e. Number of States: 3, 5, 7, 12)
# (i.e. Feature Set: features_ground, features_norm, 
#  features_polar, features_delta)
model, logL = train_a_word(my_testword, 12, features_ground)
show_model_stats(my_testword, model)
print("logL = {}".format(logL))

##### Visualize the hidden states

* Plot the Means and Variances for each State and Feature.  
* Experiment by varying the Number of States Trained for the HMM model and examine the variances. 

1) Are some models are "better" than others?  

2) How can you tell?  

3) Write what you think in the classroom online.

In [ ]:
%matplotlib inline

In [ ]:
import math
from matplotlib import (cm, pyplot as plt, mlab)

def visualize(word, model):
    """ visualize the input model for a particular word """
    
    # Variance of Hidden States
    variance = np.array([np.diag(model.covars_[i]) for i in range(model.n_components)])
 
    figures = []
    for parm_idx in range(len(model.means_[0])):
        
        print("parma_idx: {} means: {} variance: {}".format(parm_idx, model.means_[:,parm_idx], variance[:,parm_idx]))
        
        # xmin = int(min(model.means_[:,parm_idx]) - max(variance[:,parm_idx]))
        # xmax = int(max(model.means_[:,parm_idx]) + max(variance[:,parm_idx]))
        
        xmin = int((min(model.means_[:,parm_idx]) - max(variance[:,parm_idx]))*100)/100
        xmax = int((max(model.means_[:,parm_idx]) + max(variance[:,parm_idx]))*100)/100
        
        fig, axs = plt.subplots(model.n_components, sharex=True, sharey=False)
        colours = cm.rainbow(np.linspace(0, 1, model.n_components))
        for i, (ax, colour) in enumerate(zip(axs, colours)):
            x = np.linspace(xmin, xmax, 100)
            mu = model.means_[i, parm_idx]
            sigma = math.sqrt(np.diag(model.covars_[i])[parm_idx])
            
            print("parma_idx: {} i: {} mu: {} sigma: {} xmin: {} xmax: {}".format(parm_idx, i, mu, sigma, xmin, xmax))
            
            ax.plot(x, mlab.normpdf(x, mu, sigma), c=colour)
            ax.set_title("{} feature {} hidden state #{}".format(word, parm_idx, i))
            ax.grid(True)
        figures.append(plt)
    for p in figures:
        p.show()
        
visualize(my_testword, model)

#####  ModelSelector class
* Review `SelectorModel` class in `my_model_selectors.py` module.  
* `SelectorModel` is designed as a Strategy Pattern for choosing different Model Selectors.  
* Subclass the `SelectorModel` to implement the following model selectors classes/functions in the `my_model_selectors.py` module. Run them from this notebook:

    *`SelectorCV `:  Log likelihood with CV
    * `SelectorBIC`: BIC 
    * `SelectorDIC`: DIC

* Train each word in the training set with a range of values for the number of Hidden States, then Score these alternatives with the Model Selector, and choose the "best" according to each strategy. 
* e.g. the simple case of training with a constant value for `n_components` can be called using the provided `SelectorConstant` subclass as follows:

In [ ]:
from my_model_selectors import SelectorConstant

# Experiment here with different feature sets defined in Part 1
# i.e. Feature Set: features_ground, features_norm, features_polar, 
#                   features_delta
training = asl.build_training(features_ground)

# Experiment here with different words contained in `training.words`
# (i.e. VEGETABLE, CHOCOLATE, BOOK, etc)
word = 'VEGETABLE'
model = SelectorConstant(training.get_all_sequences(), 
                         training.get_all_Xlengths(), 
                         word, n_constant=3).select()
print("Number of states trained in model for {} is {}"
      .format(word, model.n_components))

##### Cross-Validation (CV) folds
* Scoring the model simply using Log Likelihood calculated from feature sequences it trained on, we expect more complex models to have higher likelihoods, but doesn't inform us which would have a "better" likelihood score on unseen data.  The model will likely overfit as complexity is added.  
* Estimate the "better" Topology model using only training data by comparing scores using Cross-Validation (CV).  
* CV techniques include:
    * Break-down the training set into "folds", rotate which fold is "left out" of training.  The "left out" fold is scored. Use this as a proxy method of finding the "best"  model to use on "unseen data". 
        * e.g. Given a set of word sequences broken-down into three folds using [scikit-learn Kfold](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.KFold.html) class object. Implement `SelectorCV` to use this technique.

In [ ]:
from sklearn.model_selection import KFold

# Experiment here with different feature sets
training = asl.build_training(features_ground) 

# Experiment here with different words
word = 'CHOCOLATE' 
word_sequences = training.get_word_sequences(word)
split_method = KFold()
for cv_train_idx, cv_test_idx in split_method.split(word_sequences):
    print("'TRAIN' fold indices: {} 'TEST' fold indices: {}".format(cv_train_idx, cv_test_idx))  # view indices of the folds

**Tip:** To run `hmmlearn` training using X,lengths tuples on new folds, subsets must be combined based on the indices given for the folds.
* Helper utility is in `asl_utils` module named `combine_sequences` for this purpose.

##### Scoring models with other criterion (i.e. BIC and DIC)

* **BIC** when used to scoring model topologies will balance fit and complexity within the training set for each word.
    * **BIC equation** has a **penalty term** that penalizes "complexity" to avoid "overfitting". This avoids using **CV** in the selection process.  Refer to these [slides](http://www2.imm.dtu.dk/courses/02433/doc/ch6_slides.pdf) about this criterion and try to use formula mentioned the SelectorBIC implementation.

* **DIC** when used as a scoring model topology has **advantages** over BIC, shown by Alain Biem here: [reference](http://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.58.6208&rep=rep1&type=pdf) (and also found [here](https://pdfs.semanticscholar.org/ed3d/7c4a5f607201f3848d4c02dd9ba17c791fc2.pdf)).  
    * **DIC** scores the ability of a training set to discriminate one word against competing words. It provides a **penalty** if model likelihoods for non-matching words are too similar to model likelihoods for the correct word in the word set (rather than using a penalty term for complexity like in BIC)

<a id='part2_submission'></a>
### Model Selection Implementation Submission

Implement the following classes in the `my_model_selectors.py` module:
    * `SelectorCV`
    * `SelectorBIC`
    * `SelectorDIC`

Run the selectors on the following 5x five words ('FISH', 'BOOK', 'VEGETABLE', 'FUTURE', 'JOHN'), then answer given questions about your results.

**Tip:** The `hmmlearn` library may not be able to train or score all models.
    * Implement `try/except` contructs as necessary to eliminate non-viable models from consideration.

In [ ]:
words_to_train = ['FISH', 'BOOK', 'VEGETABLE', 'FUTURE', 'JOHN']
import timeit

In [ ]:
# TODO: Implement SelectorCV in my_model_selector.py
from my_model_selectors import SelectorCV

# Experiment here with different feature sets defined in Part 1
training = asl.build_training(features_ground)
sequences = training.get_all_sequences()
Xlengths = training.get_all_Xlengths()
for word in words_to_train:
    start = timeit.default_timer()
    model = SelectorCV(sequences, 
                       Xlengths, 
                       word,
                       min_n_components=2, 
                       max_n_components=15, 
                       random_state = 14).select()
    end = timeit.default_timer() - start
    if model is not None:
        print("Training complete for {} with {} states with time {} seconds".format(word, model.n_components, end))
    else:
        print("Training failed for {}".format(word))

In [ ]:
# DONE: Implement SelectorBIC in module my_model_selectors.py
from my_model_selectors import SelectorBIC

# Experiment here with different feature sets defined in Part 1
training = asl.build_training(features_ground)
sequences = training.get_all_sequences()
Xlengths = training.get_all_Xlengths()
for word in words_to_train:
    start = timeit.default_timer()
    model = SelectorBIC(sequences, Xlengths, word, 
                    min_n_components=2, max_n_components=15, random_state = 14).select()
    end = timeit.default_timer()-start
    if model is not None:
        print("Training complete for {} with {} states with time {} seconds".format(word, model.n_components, end))
    else:
        print("Training failed for {}".format(word))

In [ ]:
# TODO: Implement SelectorDIC in module my_model_selectors.py
from my_model_selectors import SelectorDIC

training = asl.build_training(features_ground)  # Experiment here with different feature sets defined in part 1
sequences = training.get_all_sequences()
Xlengths = training.get_all_Xlengths()
for word in words_to_train:
    start = timeit.default_timer()
    model = SelectorDIC(sequences, Xlengths, word, 
                    min_n_components=2, max_n_components=15, random_state = 14).select()
    end = timeit.default_timer()-start
    if model is not None:
        print("Training complete for {} with {} states with time {} seconds".format(word, model.n_components, end))
    else:
        print("Training failed for {}".format(word))

**Question 2:**  Compare and contrast the possible advantages and disadvantages of the various model selectors implemented.

**Answer 2:**

<a id='part2_test'></a>
### Model Selector Unit Testing
Run the following unit tests as a sanity check on the implemented model selectors.  The test simply looks for valid interfaces  but is not exhaustive. However, the project should not be submitted if these tests don't pass.

In [ ]:
from asl_test_model_selectors import TestSelectors
suite = unittest.TestLoader().loadTestsFromModule(TestSelectors())
unittest.TextTestRunner().run(suite)

<a id='part3_tutorial'></a>
## PART 3: Recognizer
The objective of this section is to "put it all together".  Using the four feature sets created and the three model selectors, you will experiment with the models and present your results.  Instead of training only five specific words as in the previous section, train the entire set with a feature set and model selector strategy.  
### Recognizer Tutorial
##### Train the full training set
The following example trains the entire set with the example `features_ground` and `SelectorConstant` features and model selector.  Use this pattern for you experimentation and final submission cells.



In [ ]:
# autoreload for automatically reloading changes made in my_model_selectors and my_recognizer
%load_ext autoreload
%autoreload 2

from my_model_selectors import SelectorConstant

def train_all_words(features, model_selector):
    training = asl.build_training(features)  # Experiment here with different feature sets defined in part 1
    sequences = training.get_all_sequences()
    Xlengths = training.get_all_Xlengths()
    model_dict = {}
    for word in training.words:
        model = model_selector(sequences, Xlengths, word, 
                        n_constant=3).select()
        model_dict[word]=model
    return model_dict

models = train_all_words(features_ground, SelectorConstant)
print("Number of word models returned = {}".format(len(models)))

##### Load the test set
The `build_test` method in `ASLdb` is similar to the `build_training` method already presented, but there are a few differences:
- the object is type `SinglesData` 
- the internal dictionary keys are the index of the test word rather than the word itself
- the getter methods are `get_all_sequences`, `get_all_Xlengths`, `get_item_sequences` and `get_item_Xlengths`

In [ ]:
test_set = asl.build_test(features_ground)
print("Number of test set items: {}".format(test_set.num_items))
print("Number of test set sentences: {}".format(len(test_set.sentences_index)))

<a id='part3_submission'></a>
### Recognizer Implementation Submission
For the final project submission, students must implement a recognizer following guidance in the `my_recognizer.py` module.  Experiment with the four feature sets and the three model selection methods (that's 12 possible combinations). You can add and remove cells for experimentation or run the recognizers locally in some other way during your experiments, but retain the results for your discussion.  For submission, you will provide code cells of **only three** interesting combinations for your discussion (see questions below). At least one of these should produce a word error rate of less than 60%, i.e. WER < 0.60 . 

**Tip:** The hmmlearn library may not be able to train or score all models.  Implement try/except contructs as necessary to eliminate non-viable models from consideration.

In [ ]:
# TODO implement the recognize method in my_recognizer
from my_recognizer import recognize
from asl_utils import show_errors

In [ ]:
# TODO Choose a feature set and model selector
features = features_ground # change as needed
model_selector = SelectorConstant # change as needed

# TODO Recognize the test set and display the result with the show_errors method
models = train_all_words(features, model_selector)
test_set = asl.build_test(features)
probabilities, guesses = recognize(models, test_set)
show_errors(guesses, test_set)

In [ ]:
# TODO Choose a feature set and model selector
# TODO Recognize the test set and display the result with the show_errors method

In [ ]:
# TODO Choose a feature set and model selector
# TODO Recognize the test set and display the result with the show_errors method

**Question 3:**  Summarize the error results from three combinations of features and model selectors.  What was the "best" combination and why?  What additional information might we use to improve our WER?  For more insight on improving WER, take a look at the introduction to Part 4.

**Answer 3:**

<a id='part3_test'></a>
### Recognizer Unit Tests
Run the following unit tests as a sanity check on the defined recognizer.  The test simply looks for some valid values but is not exhaustive. However, the project should not be submitted if these tests don't pass.

In [ ]:
from asl_test_recognizer import TestRecognize
suite = unittest.TestLoader().loadTestsFromModule(TestRecognize())
unittest.TextTestRunner().run(suite)

<a id='part4_info'></a>
## PART 4: (OPTIONAL)  Improve the WER with Language Models
We've squeezed just about as much as we can out of the model and still only get about 50% of the words right! Surely we can do better than that.  Probability to the rescue again in the form of [statistical language models (SLM)](https://en.wikipedia.org/wiki/Language_model).  The basic idea is that each word has some probability of occurrence within the set, and some probability that it is adjacent to specific other words. We can use that additional information to make better choices.

##### Additional reading and resources
- [Introduction to N-grams (Stanford Jurafsky slides)](https://web.stanford.edu/class/cs124/lec/languagemodeling.pdf)
- [Speech Recognition Techniques for a Sign Language Recognition System, Philippe Dreuw et al](https://www-i6.informatik.rwth-aachen.de/publications/download/154/Dreuw--2007.pdf) see the improved results of applying LM on *this* data!
- [SLM data for *this* ASL dataset](ftp://wasserstoff.informatik.rwth-aachen.de/pub/rwth-boston-104/lm/)

##### Optional challenge
The recognizer you implemented in Part 3 is equivalent to a "0-gram" SLM.  Improve the WER with the SLM data provided with the data set in the link above using "1-gram", "2-gram", and/or "3-gram" statistics. The `probabilities` data you've already calculated will be useful and can be turned into a pandas DataFrame if desired (see next cell).  
Good luck!  Share your results with the class!

In [ ]:
# create a DataFrame of log likelihoods for the test word items
df_probs = pd.DataFrame(data=probabilities)
df_probs.head()